In [1]:
import os 
os.chdir("../")
%pwd

'/home/thnhan/Documents/GIT/ireuron_end2end/wine-quality-prediction'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig():
    root_dir: Path
    data_path: Path
    target: str
    preprocessor_name: str

In [15]:
from wine_quality_project.constants import *
from wine_quality_project.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation(self)-> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            target= self.schema.TARGET_COLUMN.name,
            preprocessor_name=config.preprocessor_name
        )
        return data_transformation_config


In [17]:
from wine_quality_project import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib

In [26]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)
        train, test = train_test_split(data, test_size=0.2, random_state=42)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
        return train, test
    
    def preprocess(self, train, test):
        cols = train.columns[:-1]
        print(cols)
        target = self.config.target
        pipeline = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]
        )
        preprocessor = ColumnTransformer([('pipeline', pipeline, cols)])
        X_train, X_test = train.drop([target],axis=1), test.drop([target],axis=1)
        y_train, y_test = train[[target]], test[[target]]
        X_train = preprocessor.fit_transform(X_train)
        X_test = preprocessor.transform(X_test)
        train = pd.DataFrame(X_train, columns=cols)
        train[target] = y_train.values
        test = pd.DataFrame(X_test, columns=cols)
        test[target] = y_test.values
        return preprocessor,train, test
    
    def save2artifact(self, preprocessor, train, test):
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        joblib.dump(preprocessor, os.path.join(self.config.root_dir, self.config.preprocessor_name))


    

In [27]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation()
    data_transformation = DataTransformation(config=data_transformation_config)
    train, test = data_transformation.train_test_splitting()
    preprocessor, train, test = data_transformation.preprocess(train,test)
    data_transformation.save2artifact(preprocessor,train,test)
except Exception as e:
    raise e

[2024-04-05 22:36:03,552: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-05 22:36:03,556: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-05 22:36:03,561: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-05 22:36:03,562: INFO: common: created directory at: artifacts]
[2024-04-05 22:36:03,564: INFO: common: created directory at: artifacts/data_transformation]
[2024-04-05 22:36:03,572: INFO: 1804505512: Splitted data into training and test sets]
[2024-04-05 22:36:03,573: INFO: 1804505512: (1279, 12)]
[2024-04-05 22:36:03,574: INFO: 1804505512: (320, 12)]


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')
